In [2]:
import datetime
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [3]:
from sklearn import cross_validation
from sklearn import metrics
from sklearn import preprocessing
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

/Users/sanchez/Projects/py/Anaconda/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
import utils
import analytics
import portfolio
import backtester
import models
# import refresh
# refresh.run()

In [5]:
data = analytics.cleaned_dataset()

In [7]:
startDate = date(2017, 11, 1)
endDate = date(2017, 12, 22)
coin = "ethereum"
dates, prices, volumes = getTimeseries(coin, startDate, endDate)
actions = generateSignals(dates, prices, volumes)
plotStrategy(coin, dates, prices, volumes, actions)

NameError: name 'getTimeseries' is not defined

In [91]:
# Change color of each axis
def color_y_axis(ax, color):
    """Color your axes."""
    for t in ax.get_yticklabels():
        t.set_color(color)
    return None

def formatDatesNicely(dates):
    labels = []
    for date in dates:
        thisDate = pd.to_datetime(date)
        labels.append(thisDate)
    return labels

def formatDates(dates, nth):
    labels = []
    i = 0
    for date in dates:
        thisDate = ""
        i += 1
        if i % nth == 0:
            thisDate = pd.to_datetime(date)
            thisDate = thisDate.strftime("%d/%m/%Y")
        labels.append(thisDate)
    return labels

def plotStrategy(coin, dates, prices, volumes, actions):
    # Create axes
    fig, ax1 = plt.subplots(figsize=(20,5))
    ax2 = ax1.twinx()

    ax1.plot(formatDatesNicely(dates), prices, color='black')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Close price')

    ax2.bar(formatDatesNicely(dates), volumes, width=0.5, color='blue')
    ax2.set_ylabel('Volume')
    ax2.set_ylim(top=(max(volumes) * 3))

    color_y_axis(ax1, 'black')
    color_y_axis(ax2, 'blue')

    plt.xticks(formatDatesNicely(utils.dateRange(startDate, endDate, 1, 'd')))
    plt.format_xdata = mdates.DateFormatter('%Y-%m-%d')
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
    labels = ax1.get_xticklabels()
    plt.setp(labels, rotation=30, fontsize=10)

    for action in actions:
        if action["Action"] == "BUY" or action["Action"] == "SELL":
            colour = "green" if action["Action"] == "BUY" else "red"
            plt.axvline(x=action["Date"], color=colour, linestyle='--')
        
    plt.title(coin)
    plt.show()

def generateSignals(dates, prices, volumes):

    actions = []

    bt_dates = list(dates)
    bt_prices = list(prices)
    bt_volumes = list(volumes)

    bt_dates.reverse()
    bt_prices.reverse()
    bt_volumes.reverse()

    holding = False
    for i in range(len(bt_dates)-3):
        dt = bt_dates[i]
        px = bt_prices[i]
        volume = bt_volumes[i]

        volumeTomorrow = bt_volumes[i+1]
        volumeInTwoDays = bt_volumes[i+2]
        pxTomorrow = bt_prices[i+1]
        pxInTwoDays = bt_prices[i+2]

        signal = "HODL"

        if #volume < volumeTomorrow and volumeTomorrow < volumeInTwoDays and \
          # px < pxTomorrow and pxTomorrow < pxInTwoDays:
            px < pxTomorrow and pxTomorrow < pxInTwoDays:
            # px < pxTomorrow:
            signal = "BUY"

        if px > pxTomorrow and pxTomorrow > pxInTwoDays:
            signal = "SELL"

        actions.append({"Date": dt, "Action": signal}) 

        #print (dt.strftime("%d/%m/%Y"), px, volume, signal, volumeTomorrow, volumeInTwoDays, pxTomorrow, pxInTwoDays)
    return actions

def getTimeseries(coin, startDate, endDate):
    timeseries = data[(data["Date"] >= startDate) & 
                   (data["Date"] <= endDate) &
                   (data["Coin"] == coin)]

    bitcoin = timeseries.sort_values(by="Date", ascending=False)
    prices = list(timeseries["Close"].values)
    dates = list(timeseries["Date"].values)
    volumes = list(timeseries["Volume"].values)
    dates = [pd.to_datetime(dt).to_datetime() for dt in dates]
    return dates, prices, volumes

SyntaxError: invalid syntax (<ipython-input-91-b56f953b0c4c>, line 82)

In [101]:
def generateMLData(coin, dates, prices, volumes, actions):
    
    ml_dates = list(dates)
    ml_prices = list(prices)
    ml_volumes = list(volumes)

    ml_dates.reverse()
    ml_prices.reverse()
    ml_volumes.reverse()

    
    mlFeatures = ["PRICE", "PRICE1DAYSAGO", "PRICE2DAYSAGO", "PRICE3DAYSAGO", "PRICE4DAYSAGO", "PRICE5DAYSAGO", 
                  "VOLUME", "VOLUME1DAYSAGO", "VOLUME2DAYSAGO", "VOLUME3DAYSAGO", "VOLUME4DAYSAGO", "VOLUME5DAYSAGO"]
    mlData = []
    mlTarget = []
    
    for i in range(2, len(ml_dates)):
        dt = ml_dates[i]
        price = float(ml_prices[i])
        price1day = float(ml_prices[i-1])
        price2day = float(ml_prices[i-2])
        price3day = float(ml_prices[i-3])
        price4day = float(ml_prices[i-4])
        price5day = float(ml_prices[i-5])
        
        volume = float(ml_volumes[i])
        volume1day = float(ml_volumes[i-1])
        volume2day = float(ml_volumes[i-2])
        volume3day = float(ml_volumes[i-3])
        volume4day = float(ml_volumes[i-4])
        volume5day = float(ml_volumes[i-5])
        
        row = {"PRICE": price, "PRICE1DAYSAGO": price1day, "PRICE2DAYSAGO": price2day,
               "PRICE3DAYSAGO": price3day, "PRICE4DAYSAGO": price4day, "PRICE5DAYSAGO": price5day,
               "VOLUME": volume, "VOLUME1DAYSAGO": volume1day, "VOLUME2DAYSAGO": volume2day, 
               "VOLUME3DAYSAGO": volume3day, "VOLUME4DAYSAGO": volume4day, "VOLUME5DAYSAGO": volume5day,}
        
        tgt = 0.0
        for act in actions:
            if act["Date"] == dt:
                if act["Action"] == "BUY":
                    tgt = 1.0
                elif act["Action"] == "SELL":
                    tgt = -1.0
            
        mlData.append(row)
        mlTarget.append(tgt)
        
        #print (row, tgt)
    mlData = pd.DataFrame(mlData, columns = mlFeatures)
    return mlData, mlTarget, mlFeatures

In [102]:
startDate = date(2015, 1, 1)
endDate = date(2017, 12, 22)
coin = "ethereum"
dates, prices, volumes = getTimeseries(coin, startDate, endDate)
actions = generateSignals(dates, prices, volumes)
mlData, mlTarget, mlFeatures = generateMLData("ethereum", dates, prices, volumes, actions)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    mlData, mlTarget, test_size=0.2, random_state=42)

In [103]:
estimator = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
estimator.fit(X_train, y_train)

predictions = estimator.predict(X_test)
expected = y_test

print (estimator.score(X_test, y_test))
print (predictions)
print (expected)

0.64367816092
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[-1.0, -1.0, 0.0, 0.0, -1.0, -1.0, 1.0, 0.0, -1.0, 0.0, 1.0, -1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 1.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, 0.0, 1.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 0.0,

In [104]:
from sklearn import linear_model, datasets

In [105]:
estimator = linear_model.LogisticRegression(C=1e5)
estimator.fit(X_train, y_train)

predictions = estimator.predict(X_test)
expected = y_test

estimator.score(X_test, y_test)

0.63218390804597702

In [97]:
from sklearn.neural_network import MLPClassifier

In [98]:
estimator = MLPClassifier(hidden_layer_sizes=(30,50,30))
estimator.fit(X_train, y_train)

predictions = estimator.predict(X_test)
expected = y_test


estimator.score(X_test, y_test)
##for i in range(len(predictions)):
##    print (predictions[i], expected[i])


0.4885057471264368